In [1]:
#@Kawalya Davis
import os, shutil
import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import array_to_img

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
from PIL import Image
import cv2
import shutil

In [4]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)



In [5]:
#get working directory
os.getcwd()

NameError: name 'os' is not defined

In [4]:
os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'brucechou1983_CheXNet_Keras_0.3.0_weights.h5',
 'CNN_automated_medical_diagnostics_five_layers_chopped_two_classes_trained_on_twenty_epochs_Adam_0.0001.ipynb',
 'CNN_FATPAD_TransferLearning-EfficientNet.ipynb',
 'CNN_FATPAD_TransferLearning.ipynb',
 'Labels_FatPadDetection.xlsx',
 'Prepare_Xray_Data.ipynb',
 'README.md']

In [5]:
#change working directory to where the datasets are
root_directory=os.chdir('C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project')

In [6]:
#Provide train, validation, and test to tensorflow-keras ImageDataGenerator
base_dir_train = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_Split/train'
base_dir_validate = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_Split/validation'
base_dir_test = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_Split/test'

print(base_dir_train)

C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_Split/train


In [7]:
#run tensorflow-keras ImageDataGenerator on the directory which contains train, validate and test
train_datagen = ImageDataGenerator(
    rescale=1./255, # new parameter (maps pixel values to floats)
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest') # Note: I added this, too

# The validation data should not be augmented (just rescaled)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        base_dir_train,
        color_mode='rgb',
        target_size=(256, 256), 
        batch_size=32, 
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        base_dir_validate, 
        color_mode='rgb',
        target_size=(256, 256), 
        batch_size=32, 
        class_mode='binary')

Found 3519 images belonging to 2 classes.
Found 1005 images belonging to 2 classes.


In [8]:
#load the root model and load trained weights into it
os.chdir('C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/fatpad')
root_model = tensorflow.keras.applications.DenseNet121(weights=None,
                                     include_top=False,
                                     input_shape=(256,256,3),pooling="avg")
predictions_layer = layers.Dense(14,activation='sigmoid',name='predictions_layer')(root_model.output)
#using the functional API to add the layer defined above to the root model
root_model = tensorflow.keras.Model(inputs=root_model.input, outputs=predictions_layer)
#load weights into the root model
root_model.load_weights('./brucechou1983_CheXNet_Keras_0.3.0_weights.h5')
root_model.summary()
#chop five layers off the root_model
root_model = tensorflow.keras.Model(inputs=root_model.input, outputs=root_model.layers[-5].output)
root_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 128, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1/conv[0][0]                 
_______________________________________________________________________________________

In [9]:
#chop five layers off the root_model
root_model = tensorflow.keras.Model(inputs=root_model.input, outputs=root_model.layers[-5].output)
root_model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 128, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1/conv[0][0]                 
_______________________________________________________________________________________

In [ ]:
#freeze the conv_base of the root model
root_model.trainable = False

#create a new model whose lower layers constitute the frozen conv_base
model = models.Sequential()
model.add(root_model)

#add more layers to the new model
#you can choose any depending on the need at hand
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid',name='predictions'))
model.summary()

In [ ]:
#compile with an optimizer or loss function
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.001), metrics=['acc'])

In [ ]:
# should add smoothing to these plots
def plot_history(h):
    acc = h.history['acc']
    val_acc = h.history['val_acc']
    loss = h.history['loss']
    val_loss = h.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

In [ ]:
plot_history(history)

In [ ]:
test_generator = test_datagen.flow_from_directory(
        base_dir5_test,
        color_mode='rgb',
        target_size=(256, 256), 
        batch_size=32, 
        class_mode='binary')

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=20)
test_acc

In [ ]:
roc_curve(y_pred, test_generator.classes)  